# Paper Savior 2

Auto-explorative research with RAG critics cluster

In [1]:
# %pip install lionagi llama-index unstructured pypdf wikipedia googlesearch 'unstructured[pdf]'

you need to download `lionhub` package and install locally to run the following codes

- github link: https://github.com/lion-agi/lionhub

you can use 
`pip install -e <dir-to-unzipped-folder>`


In [2]:
# if you would like to ignore logging

# import logging
# logging.getLogger().setLevel(logging.ERROR)

In [3]:
import lionagi as li

In [4]:
topic = "Large Language Model Applications in Blockchain"
question = "Research on building a system of trust integrating Large Language Model with blockchain"
num_papers = 5

# 1. Setup

In [ ]:
from lionhub import LlamaIndex

#### a. ArXiv

In [5]:
# download papers from arxiv and read them into nodes
# if to_datanode is True, you get Lion DataNode, 
# which you can do:  DataNode.to_llama_index() & DataNode.from_llama_index()
# similar usage for langchain loaders
load_config = {
    "reader": "ArxivReader", 
    "reader_type": "llama_index", 
    "load_kwargs": {"search_query": topic, "max_results": num_papers}, 
    "to_datanode": False        
}                               

arxiv_doc_ = li.load(**load_config)

arxiv_docs = arxiv_doc_[:-num_papers]
arxiv_summary = arxiv_doc_[-num_papers:]

# chunk the papers into chunks
chunk_config = {
    "chunker": "SentenceSplitter", 
    "chunker_type": 'llama_index',
    "chunker_kwargs": {'chunk_size': 512, 'chunk_overlap':20},
    "to_datanode": False
}

arxiv_chunks = li.chunk(arxiv_docs, **chunk_config)

In [6]:
arxiv_index, arxiv_engine = LlamaIndex.vector_index(
    arxiv_chunks, rerank_=True, get_engine=True
)

arxiv_index.storage_context.persist('.storage/arvix/')

In [7]:
# you can build from storage like this, just need to find the index_id in the index_store file

# arxiv_index, arxiv_engine = LlamaIndex.vector_index(
#     from_storage=True, 
#     strorage_context_kwargs={"persist_dir": '.storage/arvix/'},
#     index_id="41ca1905-433e-418c-a0a3-5d5f912d0ac3",
#     rerank_=True, get_engine=True
# )

#### b. Textbooks

In [8]:
from pathlib import Path

book1 = Path("d2l-en.pdf")
book2 = Path("Blockchain basic.pdf")

load_config1 = {
    "reader": "UnstructuredReader", 
    "reader_type": "llama_index", 
    "load_args": [book1],
    "to_datanode": False
}

load_config2 = load_config1.copy()
load_config2.update({"load_args": [book2]})

docs1 = li.load(**load_config1)
docs2 = li.load(**load_config2)

from llama_index import Document

documents1 = [Document(text="".join([x.text for x in docs1]))]
documents2 = [Document(text="".join([x.text for x in docs2]))]

Build a triplet extraction funciton using `Transformers`

- [Transformers Doc](https://huggingface.co/docs/transformers/index)
- [Make Meaningful KG from Open Source REBEL model](https://medium.com/@haiyangli_38602/make-meaningful-knowledge-graph-from-opensource-rebel-model-6f9729a55527)

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from lionhub import KGraph

device = 'mps:0'
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
model.to(device)

def extract_triplets_wiki(text):
    kb = KGraph.text_to_wiki_kb(
        text, model=model, tokenizer=tokenizer, device=device
    )
    tubs = [(i['head'], i['type'], i['tail']) for i in kb.relations]
    return tubs

this will take quite a while, I suggest you to minimize the notebook for a couple hours, or change the source to be shorter in length

In [10]:
index_config1 = {
    "input_": documents1,
    "service_context_kwargs": {"chunk_size": 512},
    "kg_triplet_extract_fn": extract_triplets_wiki, 
    "index_kwargs": {"max_triplets_per_chunk": 2, "include_embeddings": True},
    "rerank_": True, 
    "get_engine": True,
}

d2l_index, d2l_engine = LlamaIndex.kg_index(**index_config1)
d2l_index.storage_context.persist('.storage/d2l/')

In [11]:
index_config2 = {
    "input_": documents2,
    "service_context_kwargs": {"chunk_size": 512},
    "kg_triplet_extract_fn": extract_triplets_wiki, 
    "index_kwargs": {"max_triplets_per_chunk": 2, "include_embeddings": True},
    "rerank_": True, 
    "get_engine": True,
}

bc_index, bc_engine = LlamaIndex.kg_index(**index_config2)
bc_index.storage_context.persist(persist_dir='.storage/bc/')

In [12]:
# index_config11 = {
#     "from_storage": True, 
#     "strorage_context_kwargs": {"persist_dir": '.storage/d2l/'}, 
#     "index_id": "cac9b853-6b30-4fc3-9a18-8592901df717",
#     "get_engine": True,
# }

# index_config22 = {
#     "from_storage": True, 
#     "strorage_context_kwargs": {"persist_dir": '.storage/bc/'}, 
#     "index_id": "bcd41f43-7af0-4001-a3c0-c1a753cdeaac",
#     "get_engine": True,
# }

# d2l_index, d2l_engine = LlamaIndex.kg_index(**index_config11)
# bc_index, bc_engine = LlamaIndex.kg_index(**index_config22)

#### c. google and wikipedia

In [13]:
from lionhub import Search

In [14]:
responses_google = []
responses_wiki = []

# ask gpt to write you google format docstring
def query_google(query: str):
    """
    Search Google and retrieve a natural language answer to a given query.

    Args:
        query (str): The search query to find an answer for.

    Returns:
        str: A natural language answer obtained from Google search results.

    Raises:
        Exception: If there is an issue with making the request or parsing the response.
    """
    google_agent = Search.create_google_engine()
    response = google_agent.chat(query)
    responses_google.append(response)
    return str(response)

def query_wiki(query: str):
    """
    Search Wikipedia and retrieve a natural language answer to a given query.

    Args:
        query (str): The search query to find an answer for.

    Returns:
        str: A natural language answer obtained from Google search results.

    Raises:
        Exception: If there is an issue with making the request or parsing the response.
    """
    response = Search.wiki_search(query)
    responses_wiki.append(response)
    return str(response)

#### d. make into tools

In [15]:
responses_arxiv = []
responses_d2l = []
responses_bc = []

def query_arxiv(query: str):
    """
    Query a vector index built with papers from arxiv. It takes 
    natural language query, and give natural language response. 

    Args:
        query (str): The natural language query to get an answer from the index

    Returns:
        str: The query response from index
    """
    response = arxiv_engine.query(query)
    responses_arxiv.append(response)
    
    return str(response.response)

def query_d2l(query: str):
    """
    Query a index built from machine learning textbooks. It takes 
    natural language query, and give natural language response. 

    Args:
        query (str): The natural language query to get an answer from the index

    Returns:
        str: The query response from index
    """
    response = d2l_engine.query(query)
    responses_d2l.append(response)
    return str(response.response)
        
def query_bc(query: str):
    """
    Query a index built from blockchain textbooks. It takes 
    natural language query, and give natural language response. 

    Args:
        query (str): The natural language query to get an answer from the index

    Returns:
        str: The query response from index
    """
    response = bc_engine.query(query)
    responses_bc.append(response)
    return str(response.response)

In [16]:
funcs = [query_google, query_wiki, query_arxiv, query_d2l, query_bc]
tools = li.lcall(funcs, li.func_to_tool)

# 2. Design Workflow

#### a. PROMPTS - Researcher

In [ ]:
# Prompt 1: Abstract Summary
json_format1 = {
    "summary": "Brief summary of paper abstract.",
    "core points": "Key points from the paper.",
    "relevance to research question": 
        "Explanation of paper's relevance to a specific research question."
}

# Prompt 2: Reflections and Evaluation
json_format2 = {
    **json_format1,
    "reflections": "Reflections on the feedback received and improvements made."
}

# Prompt 3: Brainstorming for Research Question
json_format3 = {
    **json_format2,
    "Paper": "Name of the paper.",
    "Authors": "List of authors.",
    "key points for further investigation": "Points to explore further.",
    "reasoning": "Reasoning behind the selection of these points."
}

# Prompt 4: Final Deliverable Presentation
json_format4 = {
    **json_format3,
    "next steps": "Proposed next steps based on the research."
}

In [17]:
# Researcher System Configuration
researcher_system = {
    "persona": "World-class researcher",
    "requirements": "Clear, precise answers with a confident tone.",
    "responsibilities": "Researching a specific topic and question.",
    "notice": "Collaboration with critics for alignment with project goals."
}

# Instruction Set for Researcher

researcher_instruct1 = {
    "task step": "1",
    "task name": "Read Paper Abstracts",
    "task objective": "Initial understanding of papers.",
    "deliverable": {
        "delivery required": "yes", 
        "format": json_format1
    }
}

researcher_instruct2 = {
    "task step": "2",
    "task name": "Reflect on Feedback",
    "task objective": "Improve understanding and relevance.",
    "deliverable": {
        "delivery required": "yes", 
        "format": json_format2
    }
}

researcher_instruct3 = {
    "task step": "3",
    "task name": "Brainstorm for Research Question",
    "task objective": "Ideas for research question assistance.",
    "deliverable": {
        "delivery required": "yes", 
        "format": json_format3
    }
}

researcher_instruct4 = {
    "task step": "4",
    "task name": "Final Deliverable Presentation",
    "task objective": "Present final research output.",
    "deliverable": {
        "delivery required": "yes", 
        "format": json_format4
    }
}


#### b. PROMPTS - critic cluster

In [18]:
def get_critic_system(name_):
    tool_manual = {
        "notice": f"""
            Use the QA bot for each task. It queries an index of {name_} 
            and allows up to 3 queries per task.
        """
    }
    system = {
        "name": f"Critic_{name_}",
        "resource": f"Resource: {name_}",
        "persona": "World-class researcher",
        "responsibilities": """
            Check the quality of plans and code. Give feedback.
        """,
        "requirements": f"""
            As Critic_{name_}, verify the accuracy of plans and claims.
        """,
        "tools": tool_manual
    }
    return system

def get_critic_stage1(name_):
    return {
        "task step": "1",
        "task name": "Verify Claim",
        "description": f"""
            Research on a specific topic using the {name_} index. Verify claims and report findings.
        """,
        "deliverable": """
            Provide a detailed report with your findings. Include references and your name.
        """
    }

def get_critic_stage2(step_num):
    return {
        "task step": str(step_num),
        "task name": "Peer-Review for Improvement",
        "description": """
            Review and argue against findings from steps 2, 3, and 4. Use facts and sources.
        """,
        "deliverable": """
            Provide detailed reviews for each critic. Include references and your name.
        """
    }

def get_critic_stage3(step_num):
    return {
        "task step": str(step_num),
        "task name": "Finalize and Review",
        "description": """
            Write your final peer review. Consider all feedback and research relevance.
        """,
        "deliverable": """
            Present a comprehensive report of 1200+ words. Include your findings and name.
        """
    }

def get_critic_config(name_, tools):
    return {
        "name": f"critic_{name_}",
        "system": get_critic_system(name_),
        'step1': get_critic_stage1(name_),
        'step2': get_critic_stage2(2),
        'step3': get_critic_stage2(3),
        'step4': get_critic_stage2(4),
        'step5': get_critic_stage3(5), 
        "tools": tools
    }

# [query_google, query_wiki, query_arxiv, query_d2l, query_bc]
critic_configs = {
    "critic_google": get_critic_config("google", tools[0]),
    "critic_wiki": get_critic_config("wiki", tools[1]),
    "critic_arxiv": get_critic_config("arxiv", tools[2]),
    "critic_d2l": get_critic_config("d2l", tools[3]),
    "critic_bc": get_critic_config("bc", tools[4])
}

#### c. write workflow

In [19]:
tool_names = ['google', 'wiki', 'arxiv', 'd2l', 'bc']
critic_names = [f"critic_{t_}" for t_ in tool_names]

In [20]:
# create a researcher session
researcher = li.Session(system=researcher_system)

# create a new branch for each critic
for idx, name_ in enumerate(critic_names):
    researcher.new_branch(
        branch_name=li.nget(critic_configs, [name_, "name"]),
        tools=li.nget(critic_configs, [name_, "tools"]), 
        system=li.nget(critic_configs, [name_, "system"])
    )

In [21]:
import pandas as pd

async def critic_workflow(context):
    researcher.new_branch('critic_chat')

    async def _inner(step_num=None, _context=None):        
        f = lambda branch_, step_: [f"{branch_}", f"step{step_}"]
        
        async def _func(name_):
            config_ = {
                'instruction': li.nget(critic_configs, f(name_, step_num)), 
                'num': 5, 'to_': name_, 'temperature': 0.3, 'tools': True,
            }
            if _context is not None:
                config_.update({'context': _context})
            try:
                out = await researcher.auto_followup(**config_)
                return out
            except:
                return 'somehow failed'

        out_ = await li.alcall(critic_names, _func)
        return li.to_list(out_, flatten=True)

    def update_group_chat():
        df_ = lambda name_: researcher.branches[name_].filter_messages_by(sender='assistant').copy()
        
        for name_ in critic_names:
            df1 = df_(name_)
            if len(df1) > 0:
                researcher.branches['critic_chat'].extend(df1)
        
        lst = li.to_list([li.to_list(
                [df_(name_) if df_(name_) is not None else None], 
                flatten=True, dropna=True
            )
            for name_ in critic_names], flatten=True, dropna=True)
        
        dfs = pd.concat(lst)
        
        researcher.branches['critic_chat'].extend(dfs)
        for name_ in critic_names:
            researcher.branches[name_].extend(dfs)
            
    await _inner(1, _context=context)
    update_group_chat()
    
    for i in range(1,5):
        await _inner(i+1)
        update_group_chat()
    
    msgs = researcher.branches['critic_chat'].messages.copy()
    researcher.delete_branch('critic_chat')
    return msgs

# 3. Run

depending on your prompts, this might take a long while as well

#### Researcher Instruction 1: read abstract

In [22]:
r_context1 = str(arxiv_summary[1])

r1 = await researcher.chat(
    instruction=researcher_instruct1, 
    context=r_context1, 
    temperature=0.3
)

In [23]:
c_context1 = {
    "paper_summary": r_context1, 
    "researcher_work": str(r1)
}

reports1 = await critic_workflow(c_context1)

Branch critic_chat is deleted.


#### Researcher Instruction 2: reflect

In [24]:
r_context2 = str(
    [i.content for _, i in reports1.iloc[-10:].iterrows()]
)

r2_1 = await researcher.chat(
    instruction=researcher_instruct2, 
    context=r_context2,
    temperature=0.5
)

r2_2 = await researcher.chat(
    instruction = "integrate critics comments, reflect on your work and try again. "
)

c_context2 = {
    "researcher_work": str(r2_2)
}

reports2 = await critic_workflow(c_context2)

Branch critic_chat is deleted.


#### Researcher Instruction 3: brainstrom

In [25]:
r_context3 = str(
    [i.content for _, i in reports1.iloc[-10:].iterrows()]
)

r3 = await researcher.chat(
    instruction=researcher_instruct3, 
    context=r_context3
)

r3_2 = await researcher.chat(
    instruction = "integrate critics comments, reflect on your work and try again. "
)

c_context3 = {
    "researcher_work": str(r3_2)
}

reports3 = await critic_workflow(c_context3)

Branch critic_chat is deleted.


#### Researcher Instruction 4: output

In [26]:
r_context4 = str(
    [i.content for _, i in reports1.iloc[-10:].iterrows()]
)

r4 = await researcher.chat(
    instruction=researcher_instruct4, 
    context=r_context4
)

r4_2 = await researcher.chat(
    instruction = "integrate critics comments, reflect on your work and present final output to user, be as long as possible"
)

# 4. Result

#### Check the overall

In [27]:
ttl_messages = [
    i.messages.copy().dropna(how='all') for _, i in researcher.branches.items() 
    if len(i.messages) > 1
]
ttl_df = pd.concat(ttl_messages).drop_duplicates(subset=['node_id', 'content'])

In [32]:
ttl_df.to_csv('all_critics_cluster.csv')

In [33]:
ttl_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396 entries, 0 to 106
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   node_id    396 non-null    object        
 1   role       396 non-null    object        
 2   sender     396 non-null    object        
 3   timestamp  396 non-null    datetime64[ns]
 4   content    396 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 18.6+ KB


In [34]:
# number of reponses from all assistants

len(ttl_df[ttl_df.sender == 'assistant'])

76

In [36]:
# number of queries answered

len(ttl_df[ttl_df.sender == 'action_response'])

94

#### check each tool uses

In [37]:
print(f"total number of arxiv queries: {len(responses_arxiv)}")
print(f"total number of d2l queries: {len(responses_d2l)}")
print(f"total number of bc queries: {len(responses_bc)}")
print(f"total number of google queries: {len(responses_google)}")
print(f"total number of wiki queries: {len(responses_wiki)}")

total number of arxiv queries: 25
total number of d2l queries: 31
total number of bc queries: 9
total number of google queries: 18
total number of wiki queries: 11


# 5. Improve outputs

In [ ]:
r5 = await researcher.chat('please be more specific in terms of technical details, use your conversation context to supplement. for example, introduce which technologies, which references should be used where and include some reasoning. you can do it')

In [44]:
print(r5)

I appreciate your encouragement and the opportunity to delve into a more technically detailed research proposal. Utilizing the context of our previous conversations, let’s articulate a comprehensive plan that integrates Large Language Models (LLMs) with blockchain technology to foster a system of trust.

**Title:**
"Design and Implementation of a Trust-Enhancing Framework via LLM-Blockchain Integration"

**Abstract:**
This research investigates the technical feasibility and implementation strategies for integrating LLMs with blockchain networks to construct a robust system of trust. By harnessing the linguistic and cognitive capabilities of LLMs and the immutable record-keeping features of blockchain, we aim to develop a platform that ensures data integrity, facilitates transparent transactions, and mitigates the risks of malicious activities across blockchain systems.

**Introduction:**
Blockchain technology has laid the foundation for creating secure and decentralized digital ledgers